In this notebook, I calculate the tiers that would minimize the Weighted Average Price for a given number of tiers in the large market case. To do that, first, I import all the required packages: 

In [1]:
import numpy as np

from scipy.stats import beta
from scipy.stats import gamma
from scipy.stats import uniform

from scipy.optimize import minimize

from scipy import optimize

Next, I write a function for selecting price from a price scheme for each sample:

In [2]:
#Function for selecting price from a price scheme for each sample.

def FPS(b,c,f,scheme):
    d=len(b) #Number of samples
    scheme = scheme[~np.isnan(scheme)] #Delete NaN value from the scheme
    Ls=len(scheme) #length of the price scheme
    
    Ns=np.zeros(d) #Number of entry for each sample
    Ps=np.ones(d) #Selected price for each sample

    for i in range(d):
        n=0 
        pn=scheme[0] 
        pn1=scheme[1] #price of the next entry
        pi=pn1-c[i]-((f[i])/b[i]) #profit of the first entry. 
        
        while (0<= pi):
            n=n+1 #Number of entries
            pn=pn1 #Price when we have n entries
            
            if (n+1) < Ls:
                pn1=scheme[n+1]
            else:
                pn1=scheme[Ls-1]
                
            pi=pn1-c[i]-((2*(n+1)*f[i])/b[i]) #profit of the next entry.
        
        Ns[i]=n
        Ps[i]=pn 

    return Ns,Ps

Setting up the parameters for the simulation:

In [3]:
B_shape, B_scale = 1.5, 10000000 #gamma function for B large country
C_alpha, C_beta = 1.3, 5 #beta function for C
F_min,F_max=30000, 500000 #min and max of the fixed cost

Random draw for each of the main variables:

In [4]:
D=1000 #Number of samples
seed=1 #seed for random generator

B = gamma.rvs(B_shape, scale=B_scale, size=D, random_state=seed) #Brand market size for large countries
C = beta.rvs(C_alpha, C_beta, size=D, random_state=seed) #Generic variable cost
F = uniform.rvs(loc=F_min, scale=(F_max-F_min), size=D, random_state=seed) #Generic entry cost, per year

Then, I write down the optimization function which we need to minimize:

In [5]:
def AveragePrice(Sch):
    Sch=np.append([1],Sch)
    NS,PS=FPS(B,C,F,Sch)
    WAP=np.sum(B*PS)/np.sum(B)
    return WAP

Next, I use the "differential evolution algorithm" to find the optimal tiers scheme for a given number of tires. We should notice that all of the tiers are between 0 and 1. Therefore I set the range for optimization parameters equal to (0,1). This algorithm is not the fastest option, but it is reliable in this case. 

In [6]:
NT=5 #Number of tiers

bounds = [ (0,1) for i in range(NT)] #Setting up the range of each tier

OpSch = optimize.differential_evolution(AveragePrice, bounds, tol=1e-4, polish=True).x #Minimization

#Printing the results:

print("Optimum price scheme:",OpSch)

print("Weighted average Price:",AveragePrice(OpSch))

Optimum price scheme: [0.5344142  0.37110395 0.2872507  0.20532636 0.12534962]
Weighted average Price: 0.34853089390965164


I also use the 'Powell' method to calculate the optimized scheme. This method is fast, but it is not reliable when the number of tiers increases: 

In [7]:
#This method is fastest, but not reliable for a bigger number of tiers.

NSch=5

Sch0=np.linspace(1, 0, NSch+2)[1:-1] #Set the initial state

method='Powell' #choose method from:{'Nelder-Mead','Powell','COBYLA'}

Results = minimize(AveragePrice, Sch0, method=method, tol=None) #Minimization

OpSch=Results.x #Save the results

print("Optimum price scheme:",OpSch)

print("Weighted average Price:",AveragePrice(OpSch))

Optimum price scheme: [0.53440897 0.39132327 0.30560658 0.21265214 0.12532067]
Weighted average Price: 0.34940011818820976
